In [1]:
import sys, os
import argparse

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../../")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../")))

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "")))

In [2]:
from fedml_api.data_preprocessing.cifar10.data_loader import load_partition_data_cifar10
from fedml_api.standalone.fedavg.my_model_trainer_classification import MyModelTrainer as MyModelTrainerCLS
from fedml_api.model.contrastive_cv.resnet_with_embedding import Resnet56
from CovaMNet2 import CovaMResnet56

import torch
from torch import nn
from collections import OrderedDict
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import random
import pickle
import wandb

# from triplet_loss import TripletLoss
from hard_triplet_loss import TripletLoss

<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject


In [3]:
dataset = 'cifar10'
data_dir = "./../../../data/cifar10"
partition_method = 'hetero'
# partition_method = 'homo'
partition_alpha = 0.5
client_num_in_total = 10
batch_size = 100
total_epochs = 500

save_model_path = 'model/cs_{0}_{1}_client_{2}_variable_covaM_epochs_{3}.pt'

device = 'cuda:3'
with_cova = True

wandb.init(
            # project="federated_nas",
            project="cova",
            name="Cova_" + str(partition_method) + "_clients" + str(client_num_in_total) + "_adam-leaky_relu_iter_with_cova" + str(
                with_cova)
        )
# config = wandb.config
# config.learning_rate = 0.01

with open(f'dataset_{partition_method}_{client_num_in_total}.pickle', 'rb') as f:
    dataset = pickle.load(f)
# [train_data_num, test_data_num, train_data_global, test_data_global, \
# #             train_data_local_num_dict, train_data_local_dict, test_data_local_dict, \
# #             class_num]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: slimfun (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
# from collections import Counter
# ls = np.array([int(l) for l in labels])
# counter_result = Counter(ls)
# print(Counter(ls))
# print(dataset[5])

In [5]:
# model = Resnet56(class_num=dataset[-1], neck='bnneck')
# model.load_state_dict(torch.load('model/cs_3_homo_client_0_oral_epochs_200.pt'))

In [6]:
with open(f'data_dict.pickle', 'rb') as f:
    data_dict = pickle.load(f)

In [7]:
cl = []
for k, v in data_dict.items():
#     print(len(v))
    cl.append(torch.stack(v).to(device))
# print(cl[0].shape)
print(len(cl))

10


In [8]:
# def cal_covariance(input):

# #     CovaMatrix_list = []
# #     mean_list = []
#     B, C, h, w = input.size()
# #     print(B)
#     support_set_sam = input.permute(1, 0, 2, 3)
#     support_set_sam = support_set_sam.contiguous().view(C, -1)
    
#     mean_support = torch.mean(support_set_sam, 1, True)
# #     mean_list.append(mean_support)
    
#     support_set_sam = support_set_sam-mean_support
    
#     covariance_matrix = support_set_sam@torch.transpose(support_set_sam, 0, 1)
#     covariance_matrix = torch.div(covariance_matrix, h*w*B-1)
    
#     return covariance_matrix.clone().detach(), mean_support.clone().detach()


# def get_cova(model, device):
#     cova_list = [0 for i in range(10)]
#     mean_list = [0 for i in range(10)]
    
#     model.to(device)
#     model.eval()
    
#     features = []
    
#     with torch.no_grad():
#         for k, v in data_dict.items():
#             x = torch.stack(data_dict[k]).to(device)
# #             print(x.shape)
#             score, feats = model(x)
#             covariance_matrix, mean_support = cal_covariance(feats.to(device))
#             cova_list[k] = covariance_matrix.to(device)
#             mean_list[k] = mean_support.to(device)
            
#     return cova_list, mean_list


In [9]:
class Client(object):
    def __init__(self, client_index, train_data_local_dict, train_data_local_num_dict, test_data_local_dict, device, model):
        self.id = client_index
        self.train_data = train_data_local_dict[self.id]
        self.local_sample_number = train_data_local_num_dict[self.id]
        self.test_local = test_data_local_dict[self.id]
        
        self.device = device
        self.model = model
model1 = CovaMResnet56(class_num=dataset[-1], neck='bnneck', with_cova=with_cova)
client_1 = Client(0, dataset[5], dataset[4], dataset[6], device, model1)

In [10]:
def test(model, test_data, device, cl, ml):
#     cl = cms[0]
#     ml = cms[1]
    
    model.to(device)
    model.eval()

    metrics = {
        'test_correct': 0,
        'test_loss': 0,
        'test_total': 0,
        'correct_cova': 0
    }

    criterion = nn.CrossEntropyLoss().to(device)

    with torch.no_grad():
        for batch_idx, (x, target) in enumerate(test_data):
            x = x.to(device)
            target = target.to(device)
#             support_ml = []
#             for l in target:
#                 support_ml.append(ml[l.data].to(device))
            pred, pred_ce_cova, feat = model(x, cl, ml=None)
            loss = criterion(pred, target)
            
#             loss_ce_cova = criterion(pred_ce_cova, target)

            _, predicted = torch.max(pred, -1)
            _, predicted_ce_cova = torch.max(pred_ce_cova, -1)
            correct = predicted.eq(target).sum()
            correct_ce_cova = predicted_ce_cova.eq(target).sum()

            metrics['test_correct'] += correct.item()
#             metrics['test_correct'] += correct_ce_cova.item()
            metrics['test_loss'] += loss.item() * target.size(0)
            metrics['test_total'] += target.size(0)
            metrics['correct_cova'] += correct_ce_cova.item()
            
    return metrics

def train_model(client, epochs):
#     learning_rate = 0.001
#     wd = 0.0001
#     learning_rate = 0.00035
#     wd = 0.0005

#     cl = cms[0]
#     ml = cms[1]
    
    
    margin = 0.3
    
    client.model.to(client.device)
    client.model.train()
    
    criterion = nn.CrossEntropyLoss().to(device)
    criterion2 = nn.CrossEntropyLoss().to(device)
    mse_loss = nn.MSELoss().to(device)
    
#     curr_lr = learning_rate
#     optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, client.model.parameters()), lr=curr_lr,
#                                          weight_decay=wd, amsgrad=True)
#     optimizer = torch.optim.SGD(client.model.parameters(), lr=0.001,
#                       momentum=0.9, weight_decay=5e-4)
    optimizer = torch.optim.Adam(client.model.parameters(), lr=0.001, betas=(0.5, 0.9))
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

    epoch_loss = []
    total_step = len(client.train_data)
#     wandb.watch(client.model, log='all')
    lbd = 0.1
    cls_it = 10000
    cova_it = 5000
    for epoch in range(epochs):
        
#         if epoch > 0:
        
#         print(ml[0])
        
        client.model.train()
        batch_loss = []
        for batch_idx, (x, labels) in enumerate(client.train_data):
            
#             cl, ml = get_cova(client.model, client.device)
#             if epoch > 0:
#                 print(cl[0])
            x, labels = x.to(device), labels.to(device)
#             print(labels.shape)
            client.model.zero_grad()
#             support_ml = []
#             for l in labels:
#                 # B, 256, 64
#                 support_ml.append(ml[l.data].to(device))
#             print(cl[0].shape)
            cls_score, cova_score, feat = client.model(x, support_imgs=cl, ml=None)
#             print(score.shape)
#             print(cova_score)
#             print(cova_score)
#             loss = criterion(cova_score, labels).to(device)
#             if epoch < 100:
            if cls_it > 0:
                loss = criterion(cls_score, labels)
                cls_it -= 1
                lbd += (1-0.1) / (epochs-100)
                loss.backward()

                optimizer.step()
                batch_loss.append(loss.item())
            elif cova_it > 0:
                loss = criterion2(cova_score, labels).to(device)
                cova_it -= 1
                
                loss.backward()

                optimizer.step()
                batch_loss.append(loss.item())
            else:
                cls_it = 10
                cova_it = 5
#             if epoch > 100 :
#             loss = criterion(cova_score, labels).to(device)
#             loss = 0.5 * loss.to(device) + criterion2(cova_score, labels).to(device)
            
        
#         print(cl[0])
            
#         if epoch % 50 == 0:
#             torch.save(client.model.state_dict(), str.format(save_model_path, client_num_in_total, partition_method, client.id, epoch))
        epoch_loss.append(sum(batch_loss) / len(batch_loss))
#         scheduler.step()
        print('Client Index = {}\tEpoch: {}\tLoss: {:.6f}'.format(
            client.id, epoch, sum(batch_loss) / len(batch_loss)))
        
        metrics = test(client.model, client.test_local, client.device, cl, None)
        test_correct = metrics['test_correct']/metrics['test_total']
        test_loss = metrics['test_loss']/metrics['test_total']
        correct_ce_cova = metrics['correct_cova']/metrics['test_total']
#         print(metrics['test_total'])
        print(f'test_correct: {test_correct}; test_loss: {test_loss}')
        wandb.log({
#             'epoch' : epoch,
            'train_loss': sum(batch_loss) / len(batch_loss),
            'test_correct' : test_correct,
            'test_loss' : test_loss,
            'correct_cova' : correct_ce_cova
        }, step=epoch)
        
    torch.save(client.model.state_dict(), str.format(save_model_path, client_num_in_total, partition_method, client.id, epochs))

In [11]:
train_model(client_1, 200)

Client Index = 0	Epoch: 0	Loss: 1.817385
test_correct: 0.23; test_loss: 2.9277112698554992
Client Index = 0	Epoch: 1	Loss: 1.515680
test_correct: 0.2826; test_loss: 2.1647622954845427
Client Index = 0	Epoch: 2	Loss: 1.419995
test_correct: 0.2748; test_loss: 2.2270856273174284
Client Index = 0	Epoch: 3	Loss: 1.365300
test_correct: 0.2851; test_loss: 2.27149911403656
Client Index = 0	Epoch: 4	Loss: 1.300049
test_correct: 0.2925; test_loss: 2.3376250851154325
Client Index = 0	Epoch: 5	Loss: 1.265135
test_correct: 0.3001; test_loss: 2.314593542814255
Client Index = 0	Epoch: 6	Loss: 1.216578
test_correct: 0.2952; test_loss: 2.4760384714603423
Client Index = 0	Epoch: 7	Loss: 1.220162
test_correct: 0.273; test_loss: 2.629383931159973
Client Index = 0	Epoch: 8	Loss: 1.160196
test_correct: 0.3149; test_loss: 2.7294832181930544
Client Index = 0	Epoch: 9	Loss: 1.130427
test_correct: 0.3386; test_loss: 2.563231053352356
Client Index = 0	Epoch: 10	Loss: 1.132578
test_correct: 0.32; test_loss: 3.268

test_correct: 0.4546; test_loss: 5.078192522525788
Client Index = 0	Epoch: 93	Loss: 0.378899
test_correct: 0.4084; test_loss: 8.205345149040221
Client Index = 0	Epoch: 94	Loss: 0.364353
test_correct: 0.4324; test_loss: 8.233255901336669
Client Index = 0	Epoch: 95	Loss: 0.372936
test_correct: 0.4662; test_loss: 5.2050770258903505
Client Index = 0	Epoch: 96	Loss: 0.365739
test_correct: 0.4319; test_loss: 5.343055169582367
Client Index = 0	Epoch: 97	Loss: 0.357265
test_correct: 0.4507; test_loss: 5.449543960094452
Client Index = 0	Epoch: 98	Loss: 0.359407
test_correct: 0.3955; test_loss: 9.381935448646546
Client Index = 0	Epoch: 99	Loss: 0.368339
test_correct: 0.3948; test_loss: 7.652058458328247
Client Index = 0	Epoch: 100	Loss: 0.348189
test_correct: 0.4199; test_loss: 9.141142849922181
Client Index = 0	Epoch: 101	Loss: 0.343167
test_correct: 0.4115; test_loss: 10.15934202671051
Client Index = 0	Epoch: 102	Loss: 0.334578
test_correct: 0.4299; test_loss: 6.096780993938446
Client Index = 

Client Index = 0	Epoch: 180	Loss: 0.204879
test_correct: 0.4952; test_loss: 4.994094173908234
Client Index = 0	Epoch: 181	Loss: 0.186294
test_correct: 0.4435; test_loss: 7.775539236068726
Client Index = 0	Epoch: 182	Loss: 0.178364
test_correct: 0.4729; test_loss: 6.965334858894348
Client Index = 0	Epoch: 183	Loss: 0.208125
test_correct: 0.4658; test_loss: 7.924437570571899
Client Index = 0	Epoch: 184	Loss: 0.195564
test_correct: 0.4522; test_loss: 9.412805271148681
Client Index = 0	Epoch: 185	Loss: 0.189428
test_correct: 0.4745; test_loss: 7.070581154823303
Client Index = 0	Epoch: 186	Loss: 0.195287
test_correct: 0.4775; test_loss: 7.264659810066223
Client Index = 0	Epoch: 187	Loss: 0.201145
test_correct: 0.4555; test_loss: 6.500175065994263
Client Index = 0	Epoch: 188	Loss: 0.196080
test_correct: 0.4705; test_loss: 6.270404002666473
Client Index = 0	Epoch: 189	Loss: 0.197129
test_correct: 0.4737; test_loss: 6.4670966482162475
Client Index = 0	Epoch: 190	Loss: 0.171016
test_correct: 0.